In [1]:
# import time
# time.sleep(2100)




---

### **4. Manejo de Datos Geoespaciales.ipynb**
  - **4.1. Introducción a niveles geográficos:** Descripción de los niveles geográficos disponibles.
  - **4.2. Carga y procesamiento de geometrías:** 
    - **4.2.1.** Importación de datos de polígonos.
    - **4.2.2.** Ajuste del Sistema de Referencia de Coordenadas (CRS).
    - **4.2.3.** Cálculo del área en km^2.
  - **4.3. Guardado de datos geoespaciales:** 
    - **4.3.1.** Funciones para guardar datos en formato GeoJSON.
    - **4.3.2.** Procesamiento y guardado de datos en diferentes niveles geográficos.

---

**CONTENIDO**

- Cargar data CS
- Resultados Complejos observables en CS
- Procesamiento para datasets geograficos
    - Mapas Ejemplo (Base Personas)
    - Comandos Mapbox CLI

In [2]:
import geopandas as gpd
import numpy as np
from funciones import generate_Qs_from_year
from datetime import datetime
import pandas as pd

# -------------------
# Parameters and Configuration
# -------------------

FRAC = 0.01
START_YEAR = 2015
END_YEAR = 2019
EXPERIMENT_TAG = 'ARG'

PATH_POBREZA = './../data/Pobreza/'


In [3]:


# Cargar el índice de precios al consumidor (CPI) desde la fuente
cpi = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi.index = pd.to_datetime(cpi.index)

# Obtener la fecha de hoy en formato año-mes
hoy = datetime.today().strftime('%Y-%m')

# Calcular el ratio de precios de hoy con respecto a los precios con índice en base al modelo
ix = cpi.loc[hoy, 'index'].values[0] / cpi.loc['2016-01', 'index'].values[0]

# Lista de columnas relacionadas con montos en pesos
columnas_pesos = ['P47T_persona', 'P47T_hogar', 'CBA', 'gap_indigencia', 'CBT', 'gap_pobreza']


In [4]:

# data = pd.read_csv('/media/matias/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.02_ARGCSactual.csv', encoding_errors='ignore')
#                     # /media/matias/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.02_ARGCSactual.csv
# ## Deflacta a precios actuales

# # Lista de fechas Q que se van a procesar
# Qs = ['2022-05-15', '2022-08-15', '2022-11-15', '2023-02-15']

# Listas para almacenar los datos consolidados de cada Q
all_info_personas = []
all_info_hogares = []

# Loop through the years
for yr in range(2021, 2023):
    yr = str(yr)  # Convert year to string for consistency with quarters
    print(f"Processing data for year {yr}...")

    # Processing data for each quarter of the specified year
    relevant_quarters = generate_Qs_from_year(yr)
    for Q in relevant_quarters:
        print(f"Procesando fecha Q: {Q}")
        
        # Nombres de los archivos dependientes de Q
        personas_ingresos_Q_file = f'{PATH_POBREZA}personas_ingresos_f{FRAC}_{Q}_{EXPERIMENT_TAG}.csv'
        pobreza_hogares_file = f'{PATH_POBREZA}pobreza_hogares_f{FRAC}_q{Q}.csv'
        hogares_geo_file = f'{PATH_POBREZA}hogares_geo_f{FRAC}_{Q.split("-")[0]}_{EXPERIMENT_TAG}.csv'

        # Cargar los archivos
        personas_ingresos_Q = pd.read_csv(personas_ingresos_Q_file)
        pobreza_hogares = pd.read_csv(pobreza_hogares_file)
        hogares_geo = pd.read_csv(hogares_geo_file)

        # Fusiones para obtener los datasets consolidados
        info_personas = personas_ingresos_Q.merge(pobreza_hogares, on=['HOGAR_REF_ID', 'Q'], how='left').merge(hogares_geo, on='HOGAR_REF_ID', how='left')
        info_hogares = pobreza_hogares.merge(hogares_geo, on='HOGAR_REF_ID', how='left')
        
        # Añadir los conjuntos de datos consolidados a las listas
        all_info_personas.append(info_personas)
        all_info_hogares.append(info_hogares)


Processing data for year 2021...
Procesando fecha Q: 2021-02-15
Procesando fecha Q: 2021-05-15
Procesando fecha Q: 2021-08-15
Procesando fecha Q: 2021-11-15
Processing data for year 2022...
Procesando fecha Q: 2022-02-15
Procesando fecha Q: 2022-05-15


KeyboardInterrupt: 

In [ ]:
 
# Concatenar todos los conjuntos de datos de las diferentes fechas
cross_section_personas = pd.concat(all_info_personas, ignore_index=True)
cross_section_hogares = pd.concat(all_info_hogares, ignore_index=True)


## Adaptar datos (Pesos actuales, AGLOS si, IDFRAC)
for df in [cross_section_personas, cross_section_hogares]:
    for col in columnas_pesos:
        if col in df.columns: 
            if col == 'P47T_persona': df[col] = np.power(10, df[col]) - 1
            df[col] = (ix*df[col]).round(-1).astype(int)
            
    # df = df.merge(radio_ref, axis = 1), on = ['RADIO_REF_ID', 'AGLOMERADO'], how = 'left')
    df['AGLOSI'] = df.AGLOMERADO != 0

    df['IDFRAC'] = df['COD_2010'].astype(str).str.zfill(9).str[:-2] + '00'


# -------------------


In [ ]:
import geopandas as gpd

def compute_area_km2(geo_df):
    """Calcula el área en km^2 de un GeoDataFrame y lo añade como una nueva columna."""
    geo_df['area_km2'] = geo_df['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)
    return geo_df

# Rutas de los archivos de polígonos
admin310_f = './../../geoespacial-censo-IGN/censos_shp_CONICET_dissolved/fracs_2010.shp'
admin210_f = './../../geoespacial-censo-IGN/censos_shp_CONICET_dissolved/dptos_2010.shp'
admin1_f = './../../geoespacial-censo-IGN/IGN_shp/ign_provincia'

# Cargar polígonos de provincias del IGN
admin1 = gpd.read_file(admin1_f)
admin1['PROV'] = admin1.IN1.astype(int)
admin1 = admin1[['PROV', 'geometry']]

# Cargar y procesar polígonos de fracciones de CONICET
admin310 = gpd.read_file(admin310_f)
admin310['IDFRAC'] = admin310.PROV_ + admin310.DEPTO_ + admin310.FRACC_ + '00'
admin310 = admin310[['IDFRAC', 'geometry']]

# Cargar y procesar polígonos de departamentos de CONICET
admin210 = gpd.read_file(admin210_f)
admin210['DPTO'] = (admin210['PROV_'] + admin210['DEPTO_']).astype(int)
admin210 = admin210[['DPTO', 'geometry']]

# Ajustar CRS para que todos los GeoDataFrames tengan el mismo CRS que admin1
admin210 = admin210.to_crs(admin1.crs)
admin310 = admin310.to_crs(admin1.crs)

# Calcular el área en km^2 para cada GeoDataFrame
admin1 = compute_area_km2(admin1)
admin210 = compute_area_km2(admin210)
admin310 = compute_area_km2(admin310)


/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/shapely/measurement.py:45: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


PROV           int64
geometry    geometry
area_km2     float64
dtype: object
DPTO           int64
geometry    geometry
area_km2     float64
dtype: object
IDFRAC        object
geometry    geometry
area_km2     float64
dtype: object


Index(['OBJECTID', 'Entidad', 'Objeto', 'FNA', 'GNA', 'NAM', 'SAG', 'FDC',
       'IN1', 'SHAPE_STAr', 'SHAPE_STLe', 'geometry'],
      dtype='object')

Index(['PROV_', 'DEPTO_', 'geometry'], dtype='object')

Index(['PROV_', 'DEPTO_', 'FRACC_', 'geometry'], dtype='object')

In [ ]:
from funciones import process_and_save
import os

In [ ]:
# Define los GeoDataFrames para cada nivel geográfico
geo_dfs = {
    'PROV': admin1,
    'DPTO': admin210,
    'IDFRAC': admin310
}

# Define los subconjuntos de datos y sus prefijos correspondientes para los nombres de los archivos
data_subsets = {
    'P': cross_section_personas,
    'M24': cross_section_personas[cross_section_personas.P03 >= 24],
    'M14': cross_section_personas[cross_section_personas.P03 <= 14],
    'M6': cross_section_personas[cross_section_personas.P03 <= 6],
    'H': cross_section_hogares  # Asumiendo que data también incluye información de hogares
}

ow = False  # Sobrescribir archivos existentes
# Procesar y guardar los datos para cada subconjunto y nivel geográfico
for filename_prefix, subset in data_subsets.items():
    for geo_level, geo_df in geo_dfs.items():
        print(f"Procesando subset '{filename_prefix}' a nivel geografico '{geo_level}'...")
        
        if not ow and os.path.exists(f'poverty_{filename_prefix}_{geo_level}_sample{FRAC}.geojson'): continue
        
        process_and_save(subset, geo_level, geo_df, filename_prefix)
        print(f"Procesamiento de subset '{filename_prefix}' a nivel geografico '{geo_level}' completado.")
    print(f"Procesamiento completo para subset '{filename_prefix}'.")
print("Todos los procesamientos han sido completados.")



Procesando subset 'P' a nivel geografico 'PROV'...
Procesamiento de subset 'P' a nivel geografico 'PROV' completado.
Procesando subset 'P' a nivel geografico 'DPTO'...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_203935/187950249.py", line 25, in <module>
    process_and_save(subset, grouper, geo_df, filename_prefix)
  File "/home/matias/repos/indice-pobreza-UBA/notebooks/funciones.py", line 547, in process_and_save
  File "/home/matias/repos/indice-pobreza-UBA/notebooks/funciones.py", line 532, in save_geojson
  File "/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/geopandas/geodataframe.py", line 1263, in to_file
    _to_file(self, filename, driver, schema, index, **kwargs)
  File "/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/geopandas/io/file.py", line 572, in _to_file
    _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
  File "/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/geopandas/io/fi

In [ ]:
process_and_save(subset, geo_level, geo_df, filename_prefix)

In [ ]:
for filename_prefix, geo_level, FRAC in :
    print(f"Procesando subset '{filename_prefix}' a nivel geografico '{geo_level}'...")
    f'pobreza_{filename_prefix}_{geo_level}_{FRAC}.geojson'
    display()
    
    process_and_save(subset, geo_level, geo_df, filename_prefix)
    print(f"Procesamiento de subset '{filename_prefix}' a nivel geografico '{geo_level}' completado.")